In [1]:
# %load imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
from datetime import datetime
CHROME_DRIVER = './chromedriver'
FIREFOX_DRIVER = './geckodriver'
import time
import re
import requests
import json
import random
import copy
import pprint


def setup_driver(headless=True, driver_type=CHROME_DRIVER) -> webdriver:
    if 'chrome' in driver_type.lower():
        options = ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)
    else:
        options = FirefoxOptions()
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Firefox(executable_path=driver_type, firefox_options=options)
    return driver


In [2]:
def get_soup(link):
    #basic function to parse the soup
    driver = setup_driver()
    driver.get(link)
    time.sleep(8)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    executing_query = "document.getElementsByClassName('popover-x-button-close icl-CloseButton')[0].click()"
    #in case a pop up appears, it closes it
    if soup.find('div',{'id':'popover-form-container'}):
        driver.execute_script(executing_query)
    print(driver.current_url)
    driver.quit()
    return soup


def parse_all_pages(page):
    #from the first page, it gets the links to all other pages..
    page_links = []
    blocks = page.find('ul',{'class':'pagination-list'}).find_all('li')
    for block in blocks:
        if block.find('a', href = True):
            incomplete_link = block.find('a')['href']
            page_links.append('https://www.indeed.com'+incomplete_link)
            
    pages = []
    #stores the soup of all the pages
    pages.append(page)
    #appends the soup for the first page before getting into the loop
    for link in set(page_links):
        pages.append(get_soup(link))
        #loops with all the links to other pages and stores their soups..
    return pages

def get_all_soups(pages):
    soups = []
    #stores the soups to each job postings, the soup from their specific 
    links_to_postings = []
    #stores all the links to the specific job postings
    for page in pages:
        method_1 = page.find('div',{'class':'mosaic mosaic-provider-jobcards mosaic-provider-hydrated'})
        method_2 = page.find('div',{'class':'jobsearch-SerpJobCard unifiedRow row result clickcard'})
        #this takes care of the two different classes used in the html pages
        if method_1:
            container = method_1
            boxes = container.find_all('a', href = True)
        elif method_2:
            container = method_2
            boxes = container.find_all('a', href = True)
        else:
            return "we don't have the job postings"
            # quits the method if the links are not found.
            
        if boxes: #proceeds only if links to different postings exists
            for box in boxes:
                first = box['href']
                if '/rc/clk?jk' in first:
                    link = first.replace('/rc/clk?','https://www.indeed.com/viewjob?')
                    links_to_postings.append(link)
                    #append all the links to the list
    for link in set(links_to_postings):
        soups.append(get_soup(link))
        #get soups for unique job links parsed from different websites
    return soups

In [ ]:
def get_titles(soup):

In [3]:
page = get_soup('https://www.indeed.com/jobs?q=%22Software+Engineer%22&l=')

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/jobs?q=%22Software+Engineer%22


In [4]:
pages = parse_all_pages(page)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=10
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=20
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=30
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=40


In [ ]:
soups = get_all_soups(pages)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/viewjob?jk=1c1cca2129ef17d6&fccid=9784ae78e9834539&vjs=3
https://www.indeed.com/viewjob?jk=6797ab67821bf99e&fccid=f490aa2d3c80bc75&vjs=3
https://www.indeed.com/viewjob?jk=0ccc9495c5e70728&fccid=4edf263374ff05e2&vjs=3
https://www.indeed.com/viewjob?jk=55a3be0a59892c9c&fccid=1b50fcfb150b1b48&vjs=3
https://www.indeed.com/viewjob?jk=25a0c3b6b7ddbb3f&fccid=dd09fe3b43125016&vjs=3
https://www.indeed.com/viewjob?jk=ca6f5836440946a9&fccid=1b50fcfb150b1b48&vjs=3
https://www.indeed.com/viewjob?jk=2ab912f23dd2ffbb&fccid=66403b30a2c0d89c&vjs=3
https://www.indeed.com/viewjob?jk=c301ab0af8d9b051&fccid=68ec8a8ee6cf5ac6&vjs=3
https://www.indeed.com/viewjob?jk=fcb7593ec9da4fcf&fccid=8b9bafd9d608c198&vjs=3


In [ ]:
sd = page.find('div',{'class':'jobsearch-SerpJobCard unifiedRow row result clickcard'})

In [ ]:
for s in sd:
    d = s.find('a', href = True)
    print(d['href'])

In [ ]:
container = page.find('div',{'class':'mosaic mosaic-provider-jobcards mosaic-provider-hydrated'})

In [ ]:
boxes = container.find_all('a', href =True)

In [ ]:
boxes[0]['href'

In [ ]:
links_to_postings = []
for box in boxes:
    first = box['href']
    if '/rc/clk?jk' in first:
        link = first.replace('/rc/clk?','https://www.indeed.com/viewjob?')
        links_to_postings.append(link)

In [ ]:
soups = []
for link in links_to_postings:
    soup = get_soup(link)
    soups.append(soup)

In [ ]:
for soup in soups:
    title = soup.find('h1',{'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
    
    original = soup.find('div',{'id':'originalJobLinkContainer'}).find('a')['href']
    print(original)